# Convert TSV Data To Parquet with Athena

In this notebook, we will show you how you can easily convert that data now into Apache Parquet file format.

<img src="img/athena_convert_parquet.png" width="60%" align="left">

In [3]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
ingest_create_athena_table_parquet_passed = False

In [5]:
%store -r ingest_create_athena_table_tsv_passed

In [6]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(ingest_create_athena_table_tsv_passed)

True


In [8]:
if not ingest_create_athena_table_tsv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Import PyAthena

In [9]:
from pyathena import connect

# Create Parquet Files from TSV Table

As you can see from the query below, we’re also adding a new `year` column to our dataset by converting the `review_date` string to a date format, and then cast the year out of the date. Let’s store the year value as an integer. And let's partition the Parquet data by `Product Category`.

In [10]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/amazon-reviews-pds/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoaws"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [18]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir).cursor()

In [1]:
# conn_str = (
#     "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
#     "athena.{region_name}.amazonaws.com:443/"
#     "{schema_name}?s3_staging_dir={s3_staging_dir}&work_group=primary"
# )


# # Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
# engine = create_engine(
#     conn_str.format(
#         aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
#         aws_secret_access_key=quote_plus(AWS_SECRET_KEY),
#         region_name=AWS_REGION,
#         schema_name=SCHEMA_NAME,
#         s3_staging_dir=quote_plus(S3_STAGING_DIR),
#     )
# )
# conn = engine.connect()

# Execute Statement
_This can take a few minutes.  Please be patient._

In [13]:
import pandas as pd

# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv
)

print(statement)

CREATE TABLE IF NOT EXISTS dsoaws.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-992382405090/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM dsoaws.amazon_reviews_tsv


In [21]:
conn.execute(statement)
conn.fetchall()

[]

# Load partitions by running `MSCK REPAIR TABLE`

As a last step, we need to load the Parquet partitions. To do so, just issue the following SQL command: 

In [15]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoaws.amazon_reviews_parquet


In [22]:
import pandas as pd

conn.execute(statement)
conn.fetchall()

[]

# Show the Partitions

In [23]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoaws.amazon_reviews_parquet


In [26]:
conn.execute(statement)
# conn.description
conn.fetchall()

[('product_category=Digital_Video_Games',),
 ('product_category=Gift Card',),
 ('product_category=Digital_Software',)]

# Show the Tables

In [27]:
statement = "SHOW TABLES in {}".format(database_name)

In [37]:
conn.execute(statement)
# conn.description
df_tables = conn.fetchall()
df_tables = pd.DataFrame(df_tables)
df_tables.head()

,0
0,amazon_reviews_parquet
1,amazon_reviews_tsv


In [35]:
if table_name_parquet in df_tables.values:
    ingest_create_athena_table_parquet_passed = True

In [36]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Run Sample Query

In [38]:
product_category = "Digital_Software"

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(
    database_name, table_name_parquet, product_category
)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100


In [39]:
conn.execute(statement)
# conn.description
df = conn.fetchall()
df = pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],4,0,0,N,Y,Four Stars,So far so good,2015,2015-08-31,Digital_Software
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015,2015-08-31,Digital_Software
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],1,1,2,N,Y,One Star,Please cancel.,2015,2015-08-31,Digital_Software
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015,2015-08-31,Digital_Software
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015,2015-08-31,Digital_Software


In [40]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN CONVERTED TO PARQUET. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [42]:
from IPython.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

In just a few steps we have set up Amazon Athena to connect to our Amazon Customer Reviews TSV files, and transformed them into Apache Parquet file format. 

You might have noticed that our second sample query finished in a fraction of the time compared to the one before we ran on the TSV table. We sped up our query results by leveraging our data being stored as Parquet and partitioned by `product_category`. 


# Store Variables for the Next Notebooks

In [43]:
%store

Stored variables and their in-db values:
USE_FULL_MOVIELENS                                    -> False
bucket_name                                           -> '992382405090personalizepocvod'
comprehend_endpoint_arn                               -> 'arn:aws:comprehend:us-east-1:992382405090:documen
comprehend_train_s3_uri                               -> 's3://sagemaker-us-east-1-992382405090/data/amazon
comprehend_training_job_arn                           -> 'arn:aws:comprehend:us-east-1:992382405090:documen
data_dir                                              -> 'poc_data'
dataset_dir                                           -> 'poc_data/ml-latest-small/'
dataset_group_arn                                     -> 'arn:aws:personalize:us-east-1:992382405090:datase
forecast_arn                                          -> 'arn:aws:forecast:us-east-1:992382405090:forecast/
forecast_dataset_arn                                  -> 'arn:aws:forecast:us-east-1:992382405090:dataset/u
foreca

# Release Resources

In [45]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [44]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

# Convert TSV Data To Parquet with Athena

In this notebook, we will show you how you can easily convert that data now into Apache Parquet file format.

<img src="img/athena_convert_parquet.png" width="60%" align="left">

In [3]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
ingest_create_athena_table_parquet_passed = False

In [5]:
%store -r ingest_create_athena_table_tsv_passed

In [6]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(ingest_create_athena_table_tsv_passed)

True


In [8]:
if not ingest_create_athena_table_tsv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Import PyAthena

In [9]:
from pyathena import connect

# Create Parquet Files from TSV Table

As you can see from the query below, we’re also adding a new `year` column to our dataset by converting the `review_date` string to a date format, and then cast the year out of the date. Let’s store the year value as an integer. And let's partition the Parquet data by `Product Category`.

In [10]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/amazon-reviews-pds/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoaws"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [18]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir).cursor()

In [1]:
# conn_str = (
#     "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
#     "athena.{region_name}.amazonaws.com:443/"
#     "{schema_name}?s3_staging_dir={s3_staging_dir}&work_group=primary"
# )


# # Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
# engine = create_engine(
#     conn_str.format(
#         aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
#         aws_secret_access_key=quote_plus(AWS_SECRET_KEY),
#         region_name=AWS_REGION,
#         schema_name=SCHEMA_NAME,
#         s3_staging_dir=quote_plus(S3_STAGING_DIR),
#     )
# )
# conn = engine.connect()

# Execute Statement
_This can take a few minutes.  Please be patient._

In [13]:
import pandas as pd

# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv
)

print(statement)

CREATE TABLE IF NOT EXISTS dsoaws.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-992382405090/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM dsoaws.amazon_reviews_tsv


In [21]:
conn.execute(statement)
conn.fetchall()

[]

# Load partitions by running `MSCK REPAIR TABLE`

As a last step, we need to load the Parquet partitions. To do so, just issue the following SQL command: 

In [15]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoaws.amazon_reviews_parquet


In [22]:
import pandas as pd

conn.execute(statement)
conn.fetchall()

[]

# Show the Partitions

In [23]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoaws.amazon_reviews_parquet


In [26]:
conn.execute(statement)
# conn.description
conn.fetchall()

[('product_category=Digital_Video_Games',),
 ('product_category=Gift Card',),
 ('product_category=Digital_Software',)]

# Show the Tables

In [27]:
statement = "SHOW TABLES in {}".format(database_name)

In [37]:
conn.execute(statement)
# conn.description
df_tables = conn.fetchall()
df_tables = pd.DataFrame(df_tables)
df_tables.head()

,0
0,amazon_reviews_parquet
1,amazon_reviews_tsv


In [35]:
if table_name_parquet in df_tables.values:
    ingest_create_athena_table_parquet_passed = True

In [36]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Run Sample Query

In [38]:
product_category = "Digital_Software"

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(
    database_name, table_name_parquet, product_category
)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100


In [39]:
conn.execute(statement)
# conn.description
df = conn.fetchall()
df = pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],4,0,0,N,Y,Four Stars,So far so good,2015,2015-08-31,Digital_Software
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015,2015-08-31,Digital_Software
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],1,1,2,N,Y,One Star,Please cancel.,2015,2015-08-31,Digital_Software
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015,2015-08-31,Digital_Software
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015,2015-08-31,Digital_Software


In [40]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN CONVERTED TO PARQUET. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [42]:
from IPython.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

In just a few steps we have set up Amazon Athena to connect to our Amazon Customer Reviews TSV files, and transformed them into Apache Parquet file format. 

You might have noticed that our second sample query finished in a fraction of the time compared to the one before we ran on the TSV table. We sped up our query results by leveraging our data being stored as Parquet and partitioned by `product_category`. 


# Store Variables for the Next Notebooks

In [43]:
%store

Stored variables and their in-db values:
USE_FULL_MOVIELENS                                    -> False
bucket_name                                           -> '992382405090personalizepocvod'
comprehend_endpoint_arn                               -> 'arn:aws:comprehend:us-east-1:992382405090:documen
comprehend_train_s3_uri                               -> 's3://sagemaker-us-east-1-992382405090/data/amazon
comprehend_training_job_arn                           -> 'arn:aws:comprehend:us-east-1:992382405090:documen
data_dir                                              -> 'poc_data'
dataset_dir                                           -> 'poc_data/ml-latest-small/'
dataset_group_arn                                     -> 'arn:aws:personalize:us-east-1:992382405090:datase
forecast_arn                                          -> 'arn:aws:forecast:us-east-1:992382405090:forecast/
forecast_dataset_arn                                  -> 'arn:aws:forecast:us-east-1:992382405090:dataset/u
foreca

# Release Resources

In [45]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [44]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>